# EEG Source Reconstruction Pipeline

This script loads EEG epochs from an EEGLAB `.set` file, sets up a standard 64-channel montage, and computes the forward solution for source localization using the `fsaverage` template.


In [ ]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import pandas as pd

## 1. Load EEG Epochs

We load preprocessed EEG data stored in an EEGLAB `.set` file and convert it to an MNE `Epochs` object for further processing.


In [ ]:
raw_data_path = '../data/raw/'
path = raw_data_path + 'PPT1/'
input_fname = path + 's_101_Coordination.set'

In [ ]:
epochs = mne.io.read_epochs_eeglab(input_fname)
print(epochs)

# Subsample the epochs to get a smaller dataset (optional)
epochs = epochs[:5]

## 2. Set Montage

We apply the standard 64-channel BioSemi montage to ensure correct electrode positioning in 3D space. This step is crucial for accurate source localization.


In [ ]:
# set montage
montage = mne.channels.make_standard_montage('biosemi64')
epochs.set_montage(montage)

## 3. Compute Forward Solution

The forward solution maps neural sources in the brain to EEG scalp signals. We use the `fsaverage` template, which includes:
- A predefined **source space** (dipole grid on the cortex).
- A **BEM model** (boundary element model of the head).
- A standard **head-to-MRI transform** (`fsaverage`).

The forward model is computed with `mne.make_forward_solution()`, ensuring sources are at least **5 mm** away from the inner skull.


In [ ]:
SUBJECTS_DIR = mne.datasets.fetch_fsaverage()
SUBJECT = 'fsaverage'

if "fsaverage" in os.path.basename(SUBJECTS_DIR):
    SUBJECTS_DIR = os.path.dirname(SUBJECTS_DIR)  # Move one directory up
print(f"Fsaverage directory is at: {SUBJECTS_DIR}")

# Transformation file that aligns the EEG data with the MRI data
trans = 'fsaverage'  

# Source space that describes the locations of the dipoles
src = os.path.join(SUBJECTS_DIR, SUBJECT, 'bem', 'fsaverage-ico-5-src.fif') 

# Boundary Element Model that describes the volume conduction model
bem = os.path.join(SUBJECTS_DIR, SUBJECT, 'bem', 'fsaverage-5120-5120-5120-bem-sol.fif') 

In [ ]:
# Build the forward solution
fwd = mne.make_forward_solution(
    info=epochs.info,
    trans=trans,
    src=src,
    bem=bem,
    eeg=True,
    mindist=5.0,
    n_jobs=4
)

print(fwd)

# Save forward operator since it is the same for all subjects
source_path = '../data/source_reconstruction/'
fwd_fname = source_path + 'fsaverage_64_fwd.fif'
mne.write_forward_solution(fwd_fname, fwd, overwrite=True)

## 3.5 Load Forward solution


In [ ]:
fname_fwd = source_path + 'fsaverage_64_fwd.fif'
fwd = mne.read_forward_solution(fname_fwd)

## 4. Parcelation - Desikan-Killiany atlas
- 68 ROIs: 34 ROI from each hemisphere
- Named aparc.annot in MNE python fsaverage folder

In [ ]:
labels = mne.read_labels_from_annot("fsaverage", parc="aparc",
                                    subjects_dir=SUBJECTS_DIR)
labels = labels[:-1] # remove unknowns

label_names = [label.name for label in labels]
print(label_names)

# A. Simplest Source Reconstruction Pipeline

## 5. Compute Noise Covariance

In [ ]:
noise_cov = mne.compute_covariance(epochs)
noise_cov.plot(epochs.info, proj=True)


## 6. Create Inverse Operator

In [ ]:
inverse_operator = mne.minimum_norm.make_inverse_operator(
    info=epochs.info,
    forward=fwd,
    noise_cov=noise_cov,
    loose=1.0,
    depth=0.8,
    verbose=True
)

## 7. Apply the Inverse to Epochs

In [ ]:
epochs.set_eeg_reference(projection=True) # needed for inverse modelling, ignore error about filename due to selfmade info

snr = 3.0
lambda2 = 1.0 / snr**2
stcs = mne.minimum_norm.apply_inverse_epochs(
    epochs,
    inverse_operator,
    lambda2=lambda2,
    method='MNE',
    pick_ori='vector'
)

In [ ]:
for stc in stcs:
    print(stc)
    print(stc.data.shape)

In [ ]:
stc = stcs[1]
# Define plotting parameters
surfer_kwargs = dict(
    hemi="lh",
    subjects_dir=SUBJECTS_DIR,
)

# Plot surface
brain = stc.plot(**surfer_kwargs)

# Add title
brain.add_text(0.1, 0.9, "SourceEstimate", "title", font_size=16)

In [ ]:
stcs_label_ts = mne.extract_label_time_course(
    stcs, labels, fwd['src'], mode='mean')

print(f"Extracted label time courses for {len(stcs_label_ts)} epochs.")
print(f"Shape of the first epoch's ROI matrix: {stcs_label_ts[0].shape}")

In [ ]:
# Shape of the first epoch's ROI matrix: (68, 3, 1536)
# 68 labels, 3 directions, 1536 time points
# drop the direction dimension
stcs_label_ts = [np.mean(stc, axis=1) for stc in stcs_label_ts]
print(f"Shape of the first epoch's ROI matrix after dropping the direction dimension: {stcs_label_ts[0].shape}")

# plot the time series of the first label
plt.figure(figsize=(10, 5))
plt.plot(1e3 * stcs_label_ts[0].T)
plt.xlabel("Time (ms)")
plt.ylabel("Mean source amplitude")
plt.title(f"Mean source amplitude for {labels[0].name}")
plt.show()

In [ ]:
# Extra

In [ ]:



for epoch_idx in range(5):
    sensor_epoch = epochs[epoch_idx]  # in µV
    roi_epoch = roi_data[epoch_idx]        # in nAm (or arbitrary MNE units)

    roi_epoch = roi_data.mean(axis=0)
    sensor_mean = sensor_data.mean(axis=0)


    time_points = np.arange(sensor_epoch.size) / epochs.info['sfreq']
    time_ms = 1000 * time_points

    fig, axes = plt.subplots(2, 1, figsize=(10, 6), sharex=True)
    axes[0].plot(time_ms, sensor_epoch, color='b')
    axes[0].set_ylabel('Sensor amplitude (µV)')
    axes[0].set_title(f'{sensor_name} - epoch #{epoch_idx}')

    axes[1].plot(time_ms, roi_epoch, color='g')
    axes[1].set_xlabel('Time (ms)')
    axes[1].set_ylabel('Source amplitude (nAm)')
    axes[1].set_title(f'{roi_name} - epoch #{epoch_idx}')

    plt.tight_layout()
    plt.show()


In [ ]:

# label_ts has shape (n_labels, total_time)
# 'labels' is a list of label objects
# 'label_names' is the list of label names, or you can do [lbl.name for lbl in labels]

roi_info = mne.create_info(
    ch_names=[lbl.name for lbl in labels],    # e.g. "cuneus-lh", "insula-rh", ...
    sfreq=epochs.info['sfreq'],
    ch_types='eeg'  # treat each ROI time course as an EEG channel
)

roi_raw = mne.io.RawArray(label_ts, roi_info)
roi_raw._filenames = [""]  # to avoid filename warnings


# Plot the sensor-level concatenated data
raw.plot(
    n_channels=10,   # how many channels to view at once
    scalings='auto',
    title='Sensor-level (Raw)'
)

# Plot the ROI-level data
roi_raw.plot(
    n_channels=10,
    scalings='auto',
    title='ROI-level (Raw)'
)


In [ ]:
# --- Choose an anatomically close pair ---
sensor_name = 'P1'       # Sensor channel (from raw.ch_names)
roi_name = 'superiorparietal-lh'   # ROI channel (from roi_raw.ch_names)

# --- Extract data from raw objects ---
sensor_idx = raw.ch_names.index(sensor_name)
roi_idx = roi_raw.ch_names.index(roi_name)

sensor_signal = raw.get_data(picks=[sensor_idx])[0]  # shape: (n_times,)
roi_signal = roi_raw.get_data(picks=[roi_idx])[0]    # shape: (n_times,)

# --- Time vector ---
sfreq = raw.info['sfreq']
n_times = sensor_signal.shape[0]
time = np.arange(n_times) / sfreq  # in seconds

# --- Plot side-by-side ---
fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

axes[0].plot(time, sensor_signal, color='blue')
axes[0].set_ylabel('Sensor amplitude (µV)')
axes[0].set_title(f'Sensor-level signal: {sensor_name}')

axes[1].plot(time, roi_signal, color='green')
axes[1].set_ylabel('Source amplitude (nAm)')
axes[1].set_xlabel('Time (s)')
axes[1].set_title(f'Source-level signal: {roi_name}')

plt.tight_layout()
plt.show()

In [ ]:
from scipy.stats import zscore

sensor_signal_norm = zscore(sensor_signal)
roi_signal_norm = zscore(roi_signal)

plt.figure(figsize=(12, 5))
plt.plot(time, sensor_signal_norm, label=f'{sensor_name} (Sensor)', linewidth=2)
plt.plot(time, roi_signal_norm, label=f'{roi_name} (ROI)', linewidth=2)
plt.xlabel('Time (s)')
plt.ylabel('Z-scored amplitude')
plt.title('Normalized ROI vs Sensor waveforms')
plt.legend()
plt.show()
